In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,LSTM,Embedding,Conv1D,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

In [2]:
train_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/train.tsv', sep='\t')
train_data.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,sentiment,feedback
0,30-Jul-18,02:16:00,5138527,Theresa Diaz,"2373 Long Mews\nSouth Lorraine, FL 83715",7445448464,Black Plus,Love my new Echo Plus,5,1
1,30-Jul-18,20:13:24,7881409,Michael Wagner,"34120 James Well Apt. 930\nSouth Kenneth, AK 5...",(202)233-6662,Configuration: Fire TV Stick,Works really well.,5,1
2,29-Jul-18,03:20:48,5658396,Erin Rodriguez,"7996 Booth Mission\nTaylorport, AL 13457",(219)882-0705,Black Show,Echo Show is said to work with certain apps bu...,2,0
3,30-Jul-18,22:48:21,4317957,Christine Perez,USS Schaefer\nFPO AA 86432,291-900-6087,Heather Gray Fabric,Very cool product. Speaker sounds good with my...,5,1
4,23-Jul-18,12:36:42,7570569,Jamie Davila,"000 Kevin Cliffs Suite 674\nBrennanview, NM 77690",9527296610,Black Plus,So far I love it. It was easy to set up - if ...,5,1


In [3]:
train_data.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    1
sentiment           0
feedback            0
dtype: int64

In [4]:
train_data.fillna({"verified_reviews":"no review"}, inplace=True)

In [5]:
y=train_data['sentiment']
y = tf.keras.utils.to_categorical(y, num_classes=6)
x=train_data['verified_reviews']

In [6]:
def remove_punctuation(text):
    # Replace punctuation with a space
    cleaned_text = ''.join([char if char.isalnum() or char.isspace() else ' ' for char in text])
    return cleaned_text

cleaned_data = [remove_punctuation(text) for text in x]

In [7]:
max_vocab_size=20000
tokenizer=Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(cleaned_data)
sequence_train=tokenizer.texts_to_sequences(cleaned_data)

In [8]:
V=len(tokenizer.word_index)

In [9]:
data_train=pad_sequences(sequence_train)
T=data_train.shape[1]
print(T)

557


In [10]:
D,M=20,15
i=Input(shape=(557,))
x=Embedding(V+1,120)(i)
# x=Conv1D(32,3,activation='relu')(x)
# x=Conv1D(64,3,activation='relu')(x)
# x=Conv1D(128,3,activation='relu')(x)
# x=Flatten()(x)
x=Dense(6,activation='sigmoid')(x)

model=Model(i,x)

In [11]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(data_train,y,epochs=10)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

tfidf=TfidfVectorizer(max_features=20000,ngram_range=(1,3),analyzer='char')

In [13]:
x1=tfidf.fit_transform(cleaned_data)
y1=train_data['sentiment']

In [14]:
clf=LinearSVC(C=10,class_weight='balanced')
clf.fit(x1,y1)

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=10, class_weight='balanced')

In [15]:
test_data=pd.read_csv('/kaggle/input/anyas-gojo-revival-space-shop/test.tsv', sep='\t')
cleaned_test_data = [remove_punctuation(text) for text in test_data['verified_reviews']]
test_data1=tfidf.transform(cleaned_test_data)

In [16]:
arr=clf.predict(test_data1)
print(arr)

[5 5 1 4 5 5 5 5 1 5 1 5 4 4 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 1 5 4 5 5 5 5 5
 5 5 5 5 4 5 5 1 5 5 4 5 5 1 5 5 1 5 5 5 5 1 5 5 5 4 5 4 5 5 4 4 5 4 5 5 5
 5 2 5 5 5 5 5 5 5 5 4 5 3 5 5 5 5 1 5 5 5 4 5 5 5 5 5 4 1 4 4 5 2 5 5 4 5
 5 5 5 5 5 4 5 5 5 5 4 5 5 3 5 5 1 5 5 4 5 5 5 5 5 4 4 4 5 5 5 5 5 5 5 5 5
 5 3 5 5 4 5 5 5 5 4 5 5 5 5 5 4 5 5 5 5 5 4 5 5 5 5 4 5 5 1 5 5 5 5 5 1 5
 2 5 5 4 4 5 5 5 5 5 5 1 5 5 5 5 5 5 4 5 5 5 1 2 5 2 4 5 5 5 5 5 1 5 5 5 5
 4 5 5 5 5 5 2 5 5 5 4 5 5 5 5 5 5 1 5 5 5 5 3 5 5 4 5 4 1 5 4 5 5 5 5 5 5
 5 4 3 5 3 5 2 5 5 5 5 5 4 5 4 5 5 5 5 3 1 4 5 3 5 5 5 5 5 5 4 4 5 5 5 5 5
 5 5 5 5 5 5 2 5 5 1 5 4 1 5 5 5 5 1 1 5 5 5 5 5 4 5 5 5 5 5 5 5 5 3 5 5 5
 5 5 5 5 4 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5 5 4 1 5 5 4 4 4 4 5 5 4 5 5 5 4
 5 5 5 4 5 5 5 5 5 4 4 5 5 5 5 5 5 1 4 3 5 5 5 5 2 5 5 5 1 5 5 3 5 5 4 5 5
 5 4 5 5 3 5 5 5 5 4 5 4 5 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 2 3 5
 5 5 5 4 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 3 5 5 1 5 4 5 5 5 5 5 5 5 5 1 5 5 5
 5 5 5 3 5 5 5 5 4 2 4 5 

In [17]:
submission=pd.DataFrame({"id":test_data['id'],"sentiment":arr})
submission.to_csv("submission.csv", index=False)